INSTALL TENSORFLOW-DECISION FOREST

In [1]:
!pip install tensorflow_decision_forests -q
!pip install wurlitzer -q

     |████████████████████████████████| 15.8 MB 5.3 MB/s 
     |████████████████████████████████| 511.7 MB 4.7 kB/s 
     |████████████████████████████████| 438 kB 59.2 MB/s 
     |████████████████████████████████| 5.8 MB 46.4 MB/s 
     |████████████████████████████████| 1.6 MB 70.3 MB/s 


IMPORT LIBRARIES NEEDED

In [2]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

DOWNLOAD DATASET

In [3]:
!gdown 1dz0_0AsxbyNTJUQGWX0ZjTLcPZMfEYS7
df = pd.read_csv('/content/Busaha_Dataset.csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=1dz0_0AsxbyNTJUQGWX0ZjTLcPZMfEYS7
To: /content/Busaha_Dataset.csv
100% 45.0k/45.0k [00:00<00:00, 41.3MB/s]


,Unnamed: 0,jenis_usaha,Modal,modal_minimum,modal_maximum,tingkat_persaingan,tingkat_persaingan_idx,jumlah_pegawai,pegawai_minimum,omset,...,sosial_media,sosial_media_idx,barang_toserba,toko_toserba,toko_toserba_idx,tipe_toko,tipe_toko_idx,usaha_lain,tipe_usaha_lain,tipe_usaha_lain_idx
0,0,Transportasi,> 25jt,25000000.0,100000000.0,Sedang,1,2-4 orang,2,> 50 Juta,...,no_info,0,no_info,no_info,0,no_info,0,jasa sewa kendaraan,EX,1
1,1,Perternakan,10 - 25jt,10000000.0,25000000.0,Sedang,1,1 orang (Anda sendiri),1,10-50 Juta,...,no_info,0,no_info,no_info,0,no_info,0,no_info,no_info,0
2,2,Toserba/Toko Kelontong,1jt - 10 jt,1000000.0,10000000.0,Sedang,1,1 orang (Anda sendiri),1,< 10 Juta,...,no_info,0,Sembako,True,1,Ruko,1,no_info,no_info,0
3,3,Kuliner,1jt - 10 jt,1000000.0,10000000.0,Sedang,1,1 orang (Anda sendiri),1,< 10 Juta,...,no_info,0,no_info,no_info,0,no_info,0,no_info,no_info,0
4,4,Dropshipper,1jt - 10 jt,1000000.0,10000000.0,Rendah,0,1 orang (Anda sendiri),1,< 10 Juta,...,WA,1,no_info,no_info,0,no_info,0,no_info,no_info,0


In [6]:
print(df.shape, df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              133 non-null    int64  
 1   jenis_usaha             133 non-null    object 
 2   Modal                   133 non-null    object 
 3   modal_minimum           133 non-null    float64
 4   modal_maximum           133 non-null    float64
 5   tingkat_persaingan      133 non-null    object 
 6   tingkat_persaingan_idx  133 non-null    int64  
 7   jumlah_pegawai          133 non-null    object 
 8   pegawai_minimum         133 non-null    int64  
 9   omset                   133 non-null    object 
 10  omset_minimum           133 non-null    float64
 11  target_pasar            133 non-null    object 
 12  tp_b                    133 non-null    int64  
 13  tp_m                    133 non-null    int64  
 14  tp_a                    133 non-null    in

DROP UNNECESSARY COLUMNS

In [7]:
df = df.drop(columns=['Unnamed: 0','Modal','tingkat_persaingan','jumlah_pegawai','omset',
                      'target_pasar','lokasi','hewan_ternak','hewan_ternak_sub','lokasi_geografis',
                      'mengerti_hewan','luas_lahan','usaha_kuliner','tipe_usaha_kuliner','kemampuan_masak',
                      'barang_dropship','penyimpanan_barang','biaya_internet','sosial_media','barang_toserba',
                      'toko_toserba','tipe_toko','usaha_lain','tipe_usaha_lain'])

LABELING AND MAPPING CLASSES

In [8]:
label = "jenis_usaha"

classes = df[label].unique().tolist()
print(f"Label classes: {classes}")

df[label] = df[label].map(classes.index)

Label classes: ['Transportasi', 'Perternakan', 'Toserba/Toko Kelontong', 'Kuliner', 'Dropshipper', 'Barang Digital', 'Elektronik', 'Jasa Mengajar', 'Pakaian', 'Koleksi', 'Jasa Isi Ulang Air Minum', 'Perseroan Komanditer', 'Alat kesehatan', 'Toko Buah', 'Alat Kecantikan', 'Kosmetik', 'Penggilingan', 'Fotocopy']


SPLIT THE DATASET INTO TRAINING AND TESTING

In [10]:
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 106
No. of testing examples: 27


CONVERT INTO TENSORFLOW DATASET

In [11]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(training_data, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(testing_data, label=label)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2542: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


TRAINING THE RANDOMFOREST MODEL WITH GRADIENTBOOSTEDTREESMODEL

In [12]:
model = tfdf.keras.GradientBoostedTreesModel(num_trees=1000)
   
model.compile(metrics=["accuracy"])

model.fit(x=train_ds, validation_data=test_ds)

evaluation = model.evaluate(test_ds, return_dict=True)
print(evaluation)

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

Use /tmp/tmpznml7e1a as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.203804. Found 106 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(27, shape=(), dtype=int32)
Validation dataset read in 0:00:00.435006. Found 27 examples.
Training model...
Model trained in 0:00:02.052593
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
1/1 [==============================] - 0s 105ms/step - loss: 0.0000e+00 - accuracy: 0.9259
{'loss': 0.0, 'accuracy': 0.9259259104728699}
loss: 0.0000
accuracy: 0.9259


SAVE THE MODEL

In [13]:
model.save("/content/model")

INFO:tensorflow:Assets written to: /content/model/assets


INFO:tensorflow:Assets written to: /content/model/assets


LOAD MODEL

In [16]:
new_model = tf.keras.models.load_model('/content/model')

In [23]:
tfdf.model_plotter.plot_model_in_colab(new_model, tree_idx=0, max_depth=3)

IN TFDF FROM LOAD MODEL YOU MUST DECLARE AND CONVERT THE DATASET INTO TFDS

TESTING PREDICTION

In [17]:
pred = new_model.predict(test_ds)
np.argmax(pred, axis=1)

1/1 [==============================] - 0s 81ms/step


array([ 1,  3,  3, 12,  3, 13,  6,  2,  2,  2,  2,  2,  6,  3,  8, 15,  5,
        5,  5,  3,  2,  1,  1,  1,  1,  1,  1])

In [19]:
df_ds=tfdf.keras.pd_dataframe_to_tf_dataset(df, label=label)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2542: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [20]:
prediction = new_model.predict(df_ds)

armax = np.argmax(prediction, axis = 1)
print(armax)

1/1 [==============================] - 0s 24ms/step
[ 0  1  2  3  4  4  4  3  5  4  2  4  3  3  1  3  3  3  6  7  2  3 12  3
  9  2  2  1  3  8  1  3 10 11 12  3  2  2 13 13  4  6  6  6  6  6  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  6  6  6  6  6  4  3  3  4  1
  1  8  6  1  8  8  8 14  6  1 15 15 15  5  5  1  5  5  5  1  1  5  3  3
  3  3  3  3  3  3  3  1  1  3  3  1  2  2  3  2  3  1  3 16  3  1  2 17
  2  1  3  1  1  1  1  1  1  1  1  1  1]


In [21]:
np.testing.assert_allclose(
    model.predict(test_ds), new_model.predict(test_ds)
)

1/1 [==============================] - 0s 8ms/step


UNFORTUNATELY IN TFDF WE MUST TRAIN THE LOADED MODEL AGAIN TO PREDICT THE CLASSES IF WE USE NEW INPUT DATA 

In [24]:
new_model = tfdf.keras.GradientBoostedTreesModel()
new_model.fit(x = train_ds)

fitur = [25000000, 100000000, 1, 2, 50000000,0,1,1,1,0,0,0,0,0,0, 0,0,0,0,0,1]
dfx=pd.DataFrame (columns=["modal_minimum",
                          "modal_maximum", 
                          "tingkat_persaingan_idx", 
                          "pegawai_minimum", 
                          "omset_minimum",
                          "tp_b","tp_m","tp_a",
                          "lokasi_idx",
                          "hewan_ternak_idx",
                          "lokasi_geografis_idx",
                          "mengerti_hewan_idx",
                          "lahan_minimum",
                          "tipe_usaha_kuliner_idx",
                          "kemampuan_masak_idx",
                          "penyimpanan_barang_idx",
                          "internet_minimum",
                          "sosial_media_idx",
                          "toko_toserba_idx",
                          "tipe_toko_idx",
                          "tipe_usaha_lain_idx"])
dfx.loc[len(df)] = fitur
df_ds1=tfdf.keras.pd_dataframe_to_tf_dataset(dfx)
prediction = new_model.predict(df_ds1)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
less_indextion = np.argsort((prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
less1, less2, less3 = less_indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")
print(f"label less 3 prediksi adalah : {classes[less1]}, {classes[less2]}, {classes[less3]}")

Use /tmp/tmpgqxnnml1 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.326101. Found 106 examples.
Training model...
Model trained in 0:00:01.152734
Compiling model...


Model compiled.


1/1 [==============================] - 0s 210ms/step
top 3 prediksi adalah : [ 0  8 15]
keseluruhan prediksi adalah : [ 0  8 15  2 10 14 13  6  3 12  7 17  4  1 16  9 11  5]
prediksi paling mendekati adalah : [0]
prediksi paling tidak mendekati adalah : 5
label top 3 prediksi adalah : Transportasi, Pakaian, Kosmetik
label less 3 prediksi adalah : Barang Digital, Perseroan Komanditer, Penggilingan


In [25]:
fitur = [100000, 25000000, 1, 2, 10000000,1,1,0,0,0,0,0,0,1,1, 0,0,0,0,0,0]
dfx=pd.DataFrame (columns=["modal_minimum",
                          "modal_maximum", 
                          "tingkat_persaingan_idx", 
                          "pegawai_minimum", 
                          "omset_minimum",
                          "tp_b","tp_m","tp_a",
                          "lokasi_idx",
                          "hewan_ternak_idx",
                          "lokasi_geografis_idx",
                          "mengerti_hewan_idx",
                          "lahan_minimum",
                          "tipe_usaha_kuliner_idx",
                          "kemampuan_masak_idx",
                          "penyimpanan_barang_idx",
                          "internet_minimum",
                          "sosial_media_idx",
                          "toko_toserba_idx",
                          "tipe_toko_idx",
                          "tipe_usaha_lain_idx"])
dfx.loc[len(df)] = fitur
df_ds1=tfdf.keras.pd_dataframe_to_tf_dataset(dfx)

prediction = new_model.predict(df_ds1)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

1/1 [==============================] - 0s 49ms/step
top 3 prediksi adalah : [3 5 2]
keseluruhan prediksi adalah : [ 3  5  2  8  1  6 17 14  0 15  9 16  4 10 11 13 12  7]
prediksi paling mendekati adalah : [3]
prediksi paling tidak mendekati adalah : 7
label top 3 prediksi adalah : Kuliner, Barang Digital, Toserba/Toko Kelontong


In [26]:
!zip -r /content/TFDF_model.zip /content/model
from google.colab import files
files.download("/content/TFDF_model.zip")

  adding: content/model/ (stored 0%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 58%)
  adding: content/model/variables/variables.index (deflated 45%)
  adding: content/model/keras_metadata.pb (deflated 89%)
  adding: content/model/saved_model.pb (deflated 91%)
  adding: content/model/assets/ (stored 0%)
  adding: content/model/assets/0486d20288af4910header.pb (deflated 39%)
  adding: content/model/assets/0486d20288af4910data_spec.pb (deflated 35%)
  adding: content/model/assets/0486d20288af4910done (stored 0%)
  adding: content/model/assets/0486d20288af4910nodes-00000-of-00001 (deflated 65%)
  adding: content/model/assets/0486d20288af4910gradient_boosted_trees_header.pb (deflated 40%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>